In [1]:
import pickle

def serialize(caminho, objeto):
    outfile = open(caminho,'wb')
    pickle.dump(objeto,outfile)
    outfile.close()

def deserialize(caminho):
    infile = open(caminho,'rb')
    objeto = pickle.load(infile)
    infile.close()
    return objeto

In [2]:
def fitfunctionSmallClassic(pesos):
    argumentos ={}
    pacman = SmartAgent(pesos)
    ghostType = loadAgent('RandomGhost', True)
    textDisplay.SLEEP_TIME = 0
    argumentos['layout'] = layout.getLayout( 'smallClassic' )
    argumentos['pacman'] = pacman
    argumentos['ghosts'] = [ghostType( i+1 ) for i in range( 2 )]
    argumentos['display'] = textDisplay.NullGraphics()
    argumentos['numGames'] = 5
    argumentos['record'] = False
    argumentos['catchExceptions'] = True
    argumentos['timeout'] = 1
    
    res = runGames(**argumentos)
    scores = [game.state.getScore() for game in res]
    avgScore = sum(scores) / float(len(scores))
     
    return avgScore

In [3]:
import sys, os
os.chdir(sys.path[0]+'/search')

from pacman import runGames, loadAgent
from pacman import Directions
import pacmanAgents
from util import Queue
import textDisplay
import game
import layout
import random
import graphicsDisplay

import numpy as np 

In [4]:
class SmartAgent(game.Agent):    
    def __init__(self, pesos):
        self.pesos = pesos
        
    def getAction(self, state):
        legal = state.getLegalPacmanActions()
        if Directions.STOP in legal: legal.remove(Directions.STOP)
    
        currentState = state
        
        successors = [(state.generateSuccessor(0, action), action) for action in legal]
        evalueted =  [(self.evaluation(state, currentState), action) for state, action in successors]
        bestEvaluation = max(evalueted)[0]
        bestActions = [pair[1] for pair in evalueted if pair[0] == bestEvaluation]
        
        return random.choice(bestActions)
        
    def getNeighbors(self, pos, walls, visited):
        c, l = pos
        neighbors = []
        
        for i in range(-1,2,1):
            for j in range(-1,2,1):
                if(abs(i) != abs(j) and c+i < walls.width and c+i >= 0 and l+j < walls.height and l+j >= 0):
                    if(walls[c+i][l+j] == False and visited[c+i][l+j] == 0):
                        neighbors.append((c+i,l+j))             
        return neighbors
        
    def bfsDistance(self, state, posIni, endPos = None, food = False):
        queue = Queue()
        
        walls = state.getWalls()
        if(food):
            foods = state.getFood()
            
        visited = np.zeros((walls.width,walls.height))
        visited[posIni[0]][posIni[1]] = 1
                
        queue.push((posIni, 1))
        while(not queue.isEmpty()):
            current = queue.pop()
            pos = current[0]
            stepsUntilNow = current[1]
            
            if(food == True):
                if(foods[pos[0]][pos[1]] == True):
                    return stepsUntilNow
            else:
                if(pos[0] == endPos[0] and pos[1] == endPos[1]):
                    return stepsUntilNow
    
            visited[pos[0]][pos[1]] = 1
            possibles = self.getNeighbors(pos,walls, visited)
            for possibleMove in possibles:
                queue.push((possibleMove, stepsUntilNow+1))
        return -1
    
    def ghosts(self, state):
        ghostsPos = state.getGhostPositions()
        n_ghosts = len(ghostsPos)
        
        scared_timer = []
        positionsNonScared = []
        positionsScared = []
        
        for i in range(n_ghosts):
            scared_timer.append(state.getGhostStates()[i].scaredTimer)
            
        for i in range(n_ghosts):
            if(scared_timer[i] == 0):
                positionsNonScared.append(np.ceil(ghostsPos[i]))
            else:
                positionsScared.append(np.ceil(ghostsPos[i]))
            
        return positionsScared, positionsNonScared
    
    def nearstElement(self, state, posElement, pacmanPos):
        minDist = float('inf')
        for pos in posElement:
            dist = self.bfsDistance(state, pacmanPos, pos)
            if(dist < minDist):
                minDist = dist
        return minDist
            
    def countScaredGhosts(self, state):
        total=0
        for ghost in state.getGhostStates():
            if(ghost.scaredTimer !=0):
                total+=1
        return total
    
    def evaluation(self, state, currentState):
        
        p1 = self.pesos[0]   # peso relacionado à distancia do fantasma malvado 
        p2 = self.pesos[1]   # peso relacionado à distancia do fantasma assustado mais próximo
        p3 = self.pesos[2]   # peso relacionado a comer um fantasma assustado
        p4 = self.pesos[3]   # peso relacionado à distancia da comida mais próxima
        p5 = self.pesos[4]   # peso relacionado a comer uma comida
        p6 = self.pesos[5]   # peso relacionado à distancia da capsula mais próxima
        p7 = self.pesos[6]   # peso relacionado a comer uma capsula
        
        score = state.getScore()
        scaredGhosts, badGhosts = self.ghosts(state)
        pacman = state.getPacmanPosition()
        capsules = state.getCapsules()
        
        avaliacao = score
        ################## Bad Ghost in next position  #################
        if(len(badGhosts) > 0):
            distBadGhost = self.nearstElement(state, badGhosts, pacman)
            if(distBadGhost <= p1):
                return float('-inf')
            
        ################## Nearst Scared Ghosts ####################
        if(len(scaredGhosts) > 0):
            distScaredGhosts = self.nearstElement(state, scaredGhosts, pacman)
            avaliacao += p2*(1/distScaredGhosts)
            
        ############### Eat scared ghost bonus #################
        numScaredCurrent = self.countScaredGhosts(currentState)
        numScared = self.countScaredGhosts(state)
        scaredDiff = numScaredCurrent - numScared
        if(scaredDiff > 0):
            avaliacao += p3 * scaredDiff
            
        ################ Nearst food ###################
        nearestFood = self.bfsDistance(state, pacman, food = True)
        avaliacao += p4*(1/nearestFood)
        
        ################ Eat food bonus ##################
        numfood= state.getFood()
        numFoodCurrent = currentState.getFood()
        foodDiff = numFoodCurrent.count(True) - numfood.count(True)
        avaliacao += p5*foodDiff
        
        ################# Nearst capsule ################
        if(len(capsules) > 0):
            distCapsule = self.nearstElement(state, capsules, pacman)
            avaliacao += p6*(1/distCapsule)
        
        ################# Eat capsule bonus ##############
        numcapsules = len(state.getCapsules())
        numCapsulesCurrentState = len(currentState.getCapsules())
        avaliacao += p7*(numCapsulesCurrentState - numcapsules)
        
        return avaliacao

In [6]:
def runGame(pesos, olayout, graphics = False):
    argumentos ={}
    pacman = SmartAgent(pesos)
    ghostType = loadAgent('RandomGhost', True)
    textDisplay.SLEEP_TIME = 0
    argumentos['layout'] = layout.getLayout(olayout)
    argumentos['pacman'] = pacman
    argumentos['ghosts'] = [ghostType( i+1 ) for i in range( 2 )]
    argumentos['display'] = textDisplay.NullGraphics()
    if(graphics):
        argumentos['display'] = graphicsDisplay.PacmanGraphics(1, frameTime = 0)
    argumentos['numGames'] = 1
    argumentos['record'] = False
    argumentos['catchExceptions'] = False
    argumentos['timeout'] = 1
    
    res = runGames(**argumentos)
    game = res[0] 

    return game

In [5]:
lista = deserialize(sys.path[0]+'/save/smallClassicData')
fitness_progress, best_individual, worst_individual, model = lista

In [16]:
melhor_individuo = model.best_solution[0]
game = runGame(melhor_individuo, 'smallClassic', graphics = True)